In [2]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout


In [3]:
async def get_html(url, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                await page.mouse.wheel(15000,15000)
                time.sleep(2)
                html = await page.innerHTML('body')
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [4]:
import pandas as pd
import time

def get_yahoo_leagues(beg,num):    
    '''
    Scrape yahoo leagues to determine league id, points format, and teams in league
    Returns a df
    '''
    league_id = []
    league_player_count = []
    league_scoring_format = []
    league_positions = []
    
    for i in range (beg,num+1):
        try:
            df = pd.read_html(f'https://football.fantasysports.yahoo.com/f1/{i}/settings') 
            df0 = df[0]
            df1 = df[1]
            league_id.append(i)
            league_scoring_format.append(df1[df1.iloc[:,0].str.contains('Receptions')].iloc[0,1])
            league_player_count.append(int(df0[df0.Setting=='Max Teams:']['Value'].iloc[0]))
            league_positions.append(df[1].iloc[:,0].name)
          
        except:
            pass

        time.sleep(7)

    return pd.DataFrame({'id':league_id,
                         'player_count':league_player_count,'format':league_scoring_format,
                         'positions':league_positions})

In [ ]:
#df = pd.read_html(f'https://football.fantasysports.yahoo.com/f1/1/settings')

In [ ]:
# df = get_yahoo_leagues(15440,15460)
# df

In [ ]:
#leagues.append(df)

In [ ]:
#save = pd.concat(leagues)

In [ ]:
#save.to_csv('league_ids.csv')

In [272]:
# Create a df from 2010-2022 with player stats and adp
adp = pd.read_csv('adp_all.csv')
adp = adp[adp.pos.isin(['RB','WR','QB','TE'])]

df = pd.read_csv('ffb_df.csv',index_col=0)

df.Player = df.Player.str.replace('*','').str.replace('+','').str.strip()
df.Player = df.Player.str.replace('II','').str.replace('III','')
adp.name = adp.name.str.replace(' II','').str.replace('GordonI','Gordon').str.strip()
adp = adp.reset_index(drop=True)

df = df.merge(adp,how='left',left_on=['Player','FantPos','year'],right_on=['name','pos','year'])

df = df[['Rk', 'Player', 'Tm', 'FantPos', 'Age', 'G', 'GS', 'Cmp', 'Att', 'Yds',
       'TD', 'Int', 'Att.1', 'Yds.1', 'Y/A', 'TD.1', 'Tgt', 'Rec', 'Yds.2',
       'Y/R', 'TD.2', 'Fmb', 'FL', 'TD.3', 'PPR', 'VBD', 'PosRank', 'year', 'adp']]

df.loc[[7323,7326,7327,7338,7340],'adp'] = [19,25,350,38,81] #Incomplete data for adp in 2012
df.loc[df[df.adp.isna()].index,'adp'] = 350

cols = ['rk','player', 'team', 'pos', 'age', 'g', 'gs', 'cmp', 'pass_att',
       'pass_yds', 'pass_tds', 'int', 'rush_att', 'rush_yard', 'y/a',
       'rush_tds', 'tgt', 'rec', 'rec_yards', 'y/r', 'rec_tds', 'fmb', 'fl',
       'rush_rec_tds', 'ppr_pts', 'vbd', 'pos_rank', 'year',
       'adp']
df.columns = cols

df = df.fillna(0)
df = df[df.pos!=0]

In [349]:
df.head()

,rk,player,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,vbd,pos_rank,year,adp
0,1,Jamaal Charles,KAN,RB,27,15,15,0.0,0.0,0.0,...,9.90,7.0,4.0,2.0,19,378.0,182.0,1,2013,2.0
1,2,LeSean McCoy,PHI,RB,25,16,16,0.0,0.0,0.0,...,10.37,2.0,1.0,1.0,11,330.6,152.0,2,2013,9.0
2,3,Peyton Manning,DEN,QB,37,16,16,450.0,659.0,5477.0,...,0.00,0.0,10.0,6.0,1,410.0,151.0,1,2013,40.0
3,4,Matt Forte,CHI,RB,28,16,16,0.0,0.0,0.0,...,8.03,3.0,2.0,2.0,12,337.3,137.0,3,2013,12.0
4,5,Jimmy Graham,NOR,TE,27,16,12,0.0,0.0,0.0,...,14.13,16.0,0.0,0.0,16,303.5,124.0,1,2013,14.0


In [350]:
# Add 'pos_rank' and 'success' columns
df = pd.read_csv('season1.csv',index_col=0)

seasons = range(2010,2023)
positions = ['QB','RB','WR','TE']
add = []

for s in seasons:
    for p in positions:
        test = df[(df.year==s)&(df.pos==p)].sort_values('adp').reset_index(drop=True)
        test['adp_by_pos'] = test.index+1
        
        test = test[(test.year==s)&(test.pos==p)].sort_values('ppr_pts',ascending=False).reset_index(drop=True)
        test['pos_rank'] = test.index+1
        
        add.append(test)

df = pd.concat(add).reset_index(drop=True)

# Add 'success' column
for i in range(0,len(df.index)):
    if df.loc[i,'pos'] =='QB' or df.loc[i,'pos'] =='TE':
        
        if df.loc[i, 'adp_by_pos'] <= 3:
            if df.loc[i,'pos_rank'] <= 3:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0
        else:
            if df.loc[i,'pos_rank']<=12:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

    else:
        if df.loc[i, 'adp_by_pos'] <= 6:
            if df.loc[i,'pos_rank'] <= 6:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

        elif df.loc[i, 'adp_by_pos'] > 36:
            if df.loc[i,'pos_rank'] < 36:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

        else:
            if df.loc[i,'pos_rank'] <= df.loc[i,'adp_by_pos']:
                df.loc[i, 'success'] = 1
            else:
                df.loc[i, 'success'] = 0
        
# Add 'round' columns
for i in range(0,len(df.index)):
    if df.loc[i,'adp'] <= 12:
        df.loc[i, 'round'] = 1
    elif df.loc[i,'adp'] <= 24:
        df.loc[i, 'round'] = 2
    elif df.loc[i,'adp'] <= 36:
        df.loc[i, 'round'] = 3
    elif df.loc[i,'adp'] <= 48:
        df.loc[i, 'round'] = 4
    elif df.loc[i,'adp'] <= 60:
        df.loc[i, 'round'] = 5
    elif df.loc[i,'adp'] <= 72:
        df.loc[i, 'round'] = 6
    elif df.loc[i,'adp'] <= 84:
        df.loc[i, 'round'] = 7
    elif df.loc[i,'adp'] <= 96:
        df.loc[i, 'round'] = 8
    elif df.loc[i,'adp'] <= 108:
        df.loc[i, 'round'] = 9
    elif df.loc[i,'adp'] <= 120:
        df.loc[i, 'round'] = 10
    elif df.loc[i,'adp'] <= 132:
        df.loc[i, 'round'] = 11
    elif df.loc[i,'adp'] <= 144:
        df.loc[i, 'round'] = 12
    elif df.loc[i,'adp'] <= 156:
        df.loc[i, 'round'] = 13
    elif df.loc[i,'adp'] <= 168:
        df.loc[i, 'round'] = 14
    else:
        df.loc[i, 'round'] = 15

In [356]:
df = pd.read_csv('season1.csv',index_col=0)

In [357]:
df.head()

,rk,player,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,fl,rush_rec_tds,ppr_pts,vbd,pos_rank,year,adp,adp_by_pos,success,round
0,12,Michael Vick,PHI,QB,30,12,12,233.0,372.0,3018.0,...,3.0,9,310.3,81.0,1,2010,350.0,54,1.0,15.0
1,17,Aaron Rodgers,GNB,QB,27,15,15,312.0,475.0,3922.0,...,1.0,4,304.5,75.0,2,2010,7.0,1,1.0,1.0
2,20,Tom Brady,NWE,QB,33,16,16,324.0,492.0,3900.0,...,1.0,1,299.0,70.0,3,2010,27.0,4,1.0,3.0
3,25,Peyton Manning,IND,QB,34,16,16,450.0,679.0,4700.0,...,1.0,0,285.8,57.0,4,2010,21.0,3,0.0,2.0
4,28,Philip Rivers,SDG,QB,29,16,16,357.0,541.0,4710.0,...,4.0,0,281.6,53.0,5,2010,48.0,7,1.0,4.0


In [343]:
df.head()

,rk,player,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,vbd,pos_rank,year,adp
0,1,Jamaal Charles,KAN,RB,27,15,15,0.0,0.0,0.0,...,9.90,7.0,4.0,2.0,19,378.0,182.0,1,2013,2.0
1,2,LeSean McCoy,PHI,RB,25,16,16,0.0,0.0,0.0,...,10.37,2.0,1.0,1.0,11,330.6,152.0,2,2013,9.0
2,3,Peyton Manning,DEN,QB,37,16,16,450.0,659.0,5477.0,...,0.00,0.0,10.0,6.0,1,410.0,151.0,1,2013,40.0
3,4,Matt Forte,CHI,RB,28,16,16,0.0,0.0,0.0,...,8.03,3.0,2.0,2.0,12,337.3,137.0,3,2013,12.0
4,5,Jimmy Graham,NOR,TE,27,16,12,0.0,0.0,0.0,...,14.13,16.0,0.0,0.0,16,303.5,124.0,1,2013,14.0


In [338]:
df[(df.year==2022)][['player','pos','rk','ppr_pts','pos_rank','adp','year']].sort_values(by='rk').head(30)

,player,pos,rk,ppr_pts,pos_rank,adp,year
5502,Patrick Mahomes,QB,1,417.4,1,50.0,2022
5503,Josh Jacobs,RB,2,328.3,1,46.0,2022
5504,Christian McCaffrey,RB,3,356.4,2,3.0,2022
5505,Derrick Henry,RB,4,302.8,3,2.0,2022
5506,Justin Jefferson,WR,5,368.7,1,6.0,2022
5507,Austin Ekeler,RB,6,372.7,4,8.0,2022
5508,Josh Allen,QB,7,395.5,2,19.0,2022
5509,Travis Kelce,TE,8,316.3,1,17.0,2022
5510,Davante Adams,WR,9,335.5,2,13.0,2022
5511,Nick Chubb,RB,10,281.4,5,7.0,2022


In [137]:
df.columns

Index(['Rk', 'Player', 'Tm', 'FantPos', 'Age', 'G', 'GS', 'Cmp', 'Att', 'Yds',
       'TD', 'Int', 'Att.1', 'Yds.1', 'Y/A', 'TD.1', 'Tgt', 'Rec', 'Yds.2',
       'Y/R', 'TD.2', 'Fmb', 'FL', 'TD.3', '2:00 PM', '2PP', 'FantPt', 'PPR',
       'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank', '-9999', 'year', 'adp',
       'name', 'pos', 'tm'],
      dtype='object')

In [234]:
df.shape

(7916, 29)

In [235]:
df.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,Y/R,TD.2,Fmb,FL,TD.3,PPR,VBD,PosRank,year,adp
0,1,Jamaal Charles,KAN,RB,27,15,15,0.0,0.0,0.0,...,9.90,7.0,4.0,2.0,19,378.0,182.0,1,2013,2.0
1,2,LeSean McCoy,PHI,RB,25,16,16,0.0,0.0,0.0,...,10.37,2.0,1.0,1.0,11,330.6,152.0,2,2013,9.0
2,3,Peyton Manning,DEN,QB,37,16,16,450.0,659.0,5477.0,...,NaN,0.0,10.0,6.0,1,410.0,151.0,1,2013,40.0
3,4,Matt Forte,CHI,RB,28,16,16,0.0,0.0,0.0,...,8.03,3.0,2.0,2.0,12,337.3,137.0,3,2013,12.0
4,5,Jimmy Graham,NOR,TE,27,16,12,0.0,0.0,0.0,...,14.13,16.0,0.0,0.0,16,303.5,124.0,1,2013,14.0


### need 'adp_by_pos', 'success', 'round'

In [164]:
df = df[~df.pos.isna()]

In [170]:
adp[adp.year==2013]

,adp,name,pos,tm,year
0,1.0,Adrian Peterson,RB,MIN,2013.0
1,2.0,Jamaal Charles,RB,KC,2013.0
2,3.0,Doug Martin,RB,TB,2013.0
3,4.0,Arian Foster,RB,HOU,2013.0
4,5.0,Calvin Johnson,WR,DET,2013.0
...,...,...,...,...,...
167,168.0,Coby Fleener,TE,IND,2013.0
168,169.0,Brian Hartline,WR,MIA,2013.0
169,170.0,Ryan Tannehill,QB,MIA,2013.0
170,171.0,Philip Rivers,QB,LAC,2013.0


In [181]:
adp.year.unique()

array([2013., 2010., 2011., 2012., 2014., 2015., 2016., 2017., 2018.,
       2019., 2020., 2021., 2022.])

In [177]:
df[(df.avg_draft_pos_ppr.isna()) & (df.rk<50)]

,rk,player,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,vbd,pos_rank,year,avg_draft_pos_ppr
23,24,Fred Jackson,BUF,RB,32,16,6,0.0,0.0,0.0,...,8.23,1.0,3.0,0.0,10,234.7,61.0,10,2013,NaN
27,28,Julius Thomas,DEN,TE,25,14,14,0.0,0.0,0.0,...,12.12,12.0,0.0,0.0,12,215.8,58.0,3,2013,NaN
30,31,Le'Veon Bell,PIT,RB,21,13,13,0.0,0.0,0.0,...,8.87,0.0,1.0,1.0,8,216.9,46.0,14,2013,NaN
43,44,Zac Stacy,STL,RB,22,14,12,0.0,0.0,0.0,...,5.42,1.0,1.0,1.0,8,183.4,31.0,18,2013,NaN
45,46,Keenan Allen,SDG,WR,21,15,14,0.0,0.0,0.0,...,14.73,8.0,2.0,2.0,8,219.6,29.0,17,2013,NaN
47,48,Julian Edelman,NWE,WR,27,16,11,0.0,0.0,0.0,...,10.06,6.0,6.0,0.0,6,251.7,27.0,18,2013,NaN
48,49,Charles Clay,MIA,TE,24,16,15,0.0,0.0,0.0,...,11.00,6.0,0.0,0.0,7,188.4,26.0,7,2013,NaN
585,2,Le'Veon Bell,PIT,RB,22,16,16,0.0,0.0,0.0,...,10.29,3.0,0.0,0.0,11,370.5,167.0,2,2014,NaN
598,15,Odell Beckham Jr.,NYG,WR,22,12,11,0.0,1.0,0.0,...,14.34,12.0,1.0,1.0,12,295.0,82.0,5,2014,NaN
599,16,Justin Forsett,BAL,RB,29,16,14,0.0,0.0,0.0,...,5.98,0.0,1.0,0.0,8,246.9,82.0,8,2014,NaN


In [179]:
adp[adp.name=='Charles Clay']

,adp,name,pos,tm,year
805,134.0,Charles Clay,TE,MIA,2014.0


In [173]:
df.shape, adp.shape

((5529, 29), (2718, 5))

In [165]:
df.isna().sum()

rk                      0
player                  0
team                    0
pos                     0
age                     0
g                       0
gs                      0
cmp                     0
pass_att                0
pass_yds                0
pass_tds                0
int                     0
rush_att                0
rush_yard               0
y/a                  2318
rush_tds                0
tgt                     2
rec                     0
rec_yards               0
y/r                   843
rec_tds                 0
fmb                     0
fl                      0
rush_rec_tds            0
ppr_pts                14
vbd                  4798
pos_rank                0
year                    0
avg_draft_pos_ppr    3589
dtype: int64

In [155]:
df.head()

,rk,player,team,pos,age,g,gs,cmp,pass_att,pass_yds,...,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,vbd,pos_rank,year,avg_draft_pos_ppr
0,1,Jamaal Charles,KAN,RB,27,15,15,0.0,0.0,0.0,...,9.90,7.0,4.0,2.0,19,378.0,182.0,1,2013,2.0
1,2,LeSean McCoy,PHI,RB,25,16,16,0.0,0.0,0.0,...,10.37,2.0,1.0,1.0,11,330.6,152.0,2,2013,9.0
2,3,Peyton Manning,DEN,QB,37,16,16,450.0,659.0,5477.0,...,NaN,0.0,10.0,6.0,1,410.0,151.0,1,2013,40.0
3,4,Matt Forte,CHI,RB,28,16,16,0.0,0.0,0.0,...,8.03,3.0,2.0,2.0,12,337.3,137.0,3,2013,12.0
4,5,Jimmy Graham,NOR,TE,27,16,12,0.0,0.0,0.0,...,14.13,16.0,0.0,0.0,16,303.5,124.0,1,2013,14.0


In [154]:
df.columns = cols

In [56]:
adp[adp.name=='Christian McCaffrey']

,adp,name,pos,tm,year
1274,24.0,Christian McCaffrey,RB,CAR,2017.0
1447,13.0,Christian McCaffrey,RB,CAR,2018.0
1634,3.0,Christian McCaffrey,RB,CAR,2019.0
1832,1.0,Christian McCaffrey,RB,CAR,2020.0
2035,1.0,Christian McCaffrey,RB,CAR,2021.0
2249,3.0,Christian McCaffrey,RB,CAR,2022.0


In [14]:
df = pd.read_csv('season1.csv',index_col=0)

In [15]:
df.columns

Index(['rk', 'player', 'team', 'pos', 'age', 'g', 'gs', 'cmp', 'pass_att',
       'pass_yds', 'pass_tds', 'int', 'rush_att', 'rush_yard', 'y/a',
       'rush_tds', 'tgt', 'rec', 'rec_yards', 'y/r', 'rec_tds', 'fmb', 'fl',
       'rush_rec_tds', 'ppr_pts', 'vbd', 'pos_rank', 'year', 'adp',
       'adp_by_pos', 'success', 'round'],
      dtype='object')

In [16]:
df = df[['player','pos','age','pos_rank','year','ppr_pts','adp','adp_by_pos','success','round']]

In [35]:
df[(df.pos=='TE')&(df.year==2020)][['player','adp','round','ppr_pts']].sort_values(by='ppr_pts',ascending=False).head(10)

,player,adp,round,ppr_pts
5827,Travis Kelce,21.0,2.0,312.8
5828,Darren Waller,64.0,6.0,278.6
5829,Logan Thomas,350.0,15.0,176.6
5830,Robert Tonyan,350.0,15.0,176.6
5831,T.J. Hockenson,149.0,13.0,175.3
5832,Mark Andrews,43.0,4.0,170.1
5833,Mike Gesicki,153.0,13.0,159.3
5834,Rob Gronkowski,72.0,6.0,149.3
5835,Noah Fant,125.0,11.0,149.3
5836,Hayden Hurst,81.0,7.0,149.1


In [36]:
df[df.pos=='TE'].groupby('adp_by_pos')['ppr_pts'].mean()

adp_by_pos
1      220.176923
2      182.669231
3      169.676923
4      146.369231
5      173.530769
          ...    
114     37.100000
115     32.100000
116     24.800000
117     55.800000
118      1.000000
Name: ppr_pts, Length: 118, dtype: float64

In [28]:
df[(df.pos!='RB')&(df.adp>4)&(df.adp<10)].sort_values('ppr_pts', ascending=False)

,player,pos,age,pos_rank,year,ppr_pts,adp,adp_by_pos,success,round
2875,Julio Jones,WR,26,2,2015,375.1,6.0,2,1.0,1.0
6782,Justin Jefferson,WR,23,1,2022,368.7,6.0,2,1.0,1.0
6193,Davante Adams,WR,29,2,2021,344.3,6.0,1,1.0,1.0
2350,Demaryius Thomas,WR,27,2,2014,340.9,7.0,2,1.0,1.0
1055,Tom Brady,QB,35,3,2012,340.3,6.0,2,1.0,1.0
4484,DeAndre Hopkins,WR,26,2,2018,333.5,8.0,2,1.0,1.0
2352,Dez Bryant,WR,26,4,2014,316.0,9.0,3,1.0,1.0
1,Aaron Rodgers,QB,27,2,2010,304.5,7.0,1,1.0,1.0
1819,Calvin Johnson,WR,28,6,2013,303.2,5.0,1,1.0,1.0
764,Roddy White,WR,30,5,2011,277.6,9.0,2,1.0,1.0


In [23]:
df.groupby(['round','pos'])['ppr_pts'].agg(['mean','min','max'])

mean    min    max
round pos                          
1.0   QB   313.450000  233.0  345.6
      RB   227.743158    9.0  471.2
      TE   249.433333  229.9  262.8
      WR   262.557143   83.9  388.2
2.0   QB   318.200000  130.8  397.4
      RB   208.938806   13.4  385.5
      TE   219.333333   97.0  316.3
      WR   238.117910   12.3  386.9
3.0   QB   287.661538  129.6  402.6
      RB   163.685484    4.5  314.8
      TE   180.291667   75.6  294.6
      WR   215.760000   65.8  335.9
4.0   QB   280.457143   99.1  410.0
      RB   160.364151   43.0  392.0
      TE   186.371429   57.1  280.3
      WR   197.304110   27.1  439.5
5.0   QB   265.372222   91.4  417.4
      RB   128.049020   19.3  250.6
      TE   144.166667   42.1  301.1
      WR   185.596610    2.7  339.2
6.0   QB   279.170968   62.4  380.8
      RB   141.728571    2.0  309.0
      TE   153.938889    8.6  294.0
      WR   174.231579    9.5  328.6
7.0   QB   228.395652   40.7  374.7
      RB   117.405769   -0.2  243.6
      TE   128.058333    9.2  209.5
      WR   159.140426   34.8  304.6
8.0   QB   265.672727   97.2  383.3
      RB   121.314286    6.8  255.6
      TE   139.870588   15.1  220.8
      WR   143.754717   15.2  339.0
9.0   QB   227.145455   12.5  415.7
      RB   136.811364   12.7  316.4
      TE   140.356250   48.5  244.4
      WR   154.850000   20.6  314.4
10.0  QB   250.309524   12.7  355.0
      RB   100.963636    0.9  222.5
      TE   145.292308   54.5  330.9
      WR   123.168000   11.4  241.9
11.0  QB   216.104167   50.5  417.1
      RB   111.265789   16.4  296.6
      TE   133.875000   20.0  214.5
      WR   133.245455    5.3  284.3
12.0  QB   192.340000   64.9  337.8
      RB    81.425000    1.9  231.4
      TE   127.291667   60.4  258.7
      WR   122.507692    2.6  283.6
13.0  QB   189.342857   30.4  347.5
      RB    86.577778    5.4  320.4
      TE   113.412500   11.3  207.2
      WR   109.134483    9.8  226.3
14.0  QB   185.250000   41.4  370.3
      RB    82.386486    0.2  280.0
      TE   122.159259    2.2  223.1
      WR   113.145946    4.3  268.9
15.0  QB    64.586517   -7.3  344.4
      RB    39.549965   -2.8  370.5
      TE    41.517145   -0.2  244.2
      WR    54.376394   -3.3  319.3

In [20]:
df[(df.adp<7)&(df.pos=='QB')]

,player,pos,age,pos_rank,year,ppr_pts,adp,adp_by_pos,success,round
1054,Aaron Rodgers,QB,29,2,2012,344.6,3.0,1,1.0,1.0
1055,Tom Brady,QB,35,3,2012,340.3,6.0,2,1.0,1.0
